In [1]:
import numpy as np
import pandas as pd
import os
from imblearn.over_sampling import SMOTE

In [2]:
# Chemins des répertoires
normal_repo = '/Users/khaled/PA/Data/normal'
anomalous_repo = '/Users/khaled/PA/Data/anormal'

def verifier_fichiers(repo):
    rapport = []
    for fichier in os.listdir(repo):
        if fichier.endswith('.csv'):
            chemin_fichier = os.path.join(repo, fichier)
            try:
                # Charger le fichier
                data = pd.read_csv(chemin_fichier)

                # Vérifications
                valeurs_manquantes = data.isnull().sum().sum()
                types_incorrects = any(data.dtypes == object)  # Vérifie si des colonnes numériques ont des types incorrects
                colonnes = data.columns.tolist()

                rapport.append({
                    'fichier': fichier,
                    'valeurs_manquantes': valeurs_manquantes,
                    'types_incorrects': types_incorrects,
                    'colonnes': colonnes
                })
            except Exception as e:
                rapport.append({'fichier': fichier, 'erreur': str(e)})
    return rapport

# Vérification des deux répertoires
rapport_normal = verifier_fichiers(normal_repo)
rapport_anormal = verifier_fichiers(anomalous_repo)

# Afficher les rapports
print("Rapport pour le traffic normal:")
for r in rapport_normal:
    print(r)

print("\nRapport pour le traffic anormal:")
for r in rapport_anormal:
    print(r)

Rapport pour le traffic normal:
{'fichier': 'part-00000-4b44f1bf-6ef8-4709-9b96-7fd0e97a8c59-c000.csv', 'valeurs_manquantes': np.int64(0), 'types_incorrects': True, 'colonnes': ['flow_key', 'start_ts', 'end_ts', 'total_bytes', 'pkt_count', 'psh_count', 'urg_count', 'fwd_bytes', 'bwd_bytes', 'fwd_pkts', 'bwd_pkts', 'dport', 'attack', 'duration_ms', 'flow_pkts_per_s', 'fwd_bwd_ratio', 'window_start', 'window_end']}
{'fichier': 'part-00000-28589e4c-32f8-487d-829c-7605ea91b2dc-c000.csv', 'valeurs_manquantes': np.int64(0), 'types_incorrects': True, 'colonnes': ['flow_key', 'start_ts', 'end_ts', 'total_bytes', 'pkt_count', 'psh_count', 'urg_count', 'fwd_bytes', 'bwd_bytes', 'fwd_pkts', 'bwd_pkts', 'dport', 'attack', 'duration_ms', 'flow_pkts_per_s', 'fwd_bwd_ratio', 'window_start', 'window_end']}
{'fichier': 'part-00000-f5d3e4d0-c43a-4833-a340-79b820b6909e-c000.csv', 'valeurs_manquantes': np.int64(0), 'types_incorrects': True, 'colonnes': ['flow_key', 'start_ts', 'end_ts', 'total_bytes', 'pk

In [3]:
def charger_fichiers(repo):
    dataframes = []
    for fichier in os.listdir(repo):
        if fichier.endswith('.csv'):
            chemin_fichier = os.path.join(repo, fichier)
            try:
                # Charger le fichier
                data = pd.read_csv(chemin_fichier)
                dataframes.append(data)
            except Exception as e:
                print(f"Erreur lors du chargement de {fichier}: {e}")
    return pd.concat(dataframes, ignore_index=True)

# Charger et concaténer les données
normal_data = charger_fichiers(normal_repo)
anomalous_data = charger_fichiers(anomalous_repo)


/var/folders/7h/ryb6h1gx2zv01ytw5bs043rw0000gp/T/ipykernel_74494/1360512867.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(dataframes, ignore_index=True)
/var/folders/7h/ryb6h1gx2zv01ytw5bs043rw0000gp/T/ipykernel_74494/1360512867.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(dataframes, ignore_index=True)


In [4]:
print(len(normal_data))
print(len(anomalous_data))

6287
69350


In [5]:
def verifier(data):
    # Vérification des valeurs manquantes
    valeurs_manquantes = data.isnull().sum()
    print("Valeurs manquantes par colonne :")
    print(valeurs_manquantes)

    # Vérification des types incorrects
    print("\nTypes des colonnes :")
    print(data.dtypes)

    # Identification des colonnes numériques
    colonnes_numeriques = data.select_dtypes(include=['float64', 'int64']).columns
    print("\nColonnes numériques :")
    print(colonnes_numeriques)

print("Vérification des données normales :")
verifier(normal_data)
print("\nVérification des données anormales :")
verifier(anomalous_data)


Vérification des données normales :
Valeurs manquantes par colonne :
flow_key           0
start_ts           0
end_ts             0
total_bytes        0
pkt_count          0
psh_count          0
urg_count          0
fwd_bytes          0
bwd_bytes          0
fwd_pkts           0
bwd_pkts           0
dport              0
attack             0
duration_ms        0
flow_pkts_per_s    0
fwd_bwd_ratio      0
window_start       0
window_end         0
dtype: int64

Types des colonnes :
flow_key            object
start_ts            object
end_ts              object
total_bytes         object
pkt_count           object
psh_count           object
urg_count           object
fwd_bytes           object
bwd_bytes           object
fwd_pkts            object
bwd_pkts            object
dport               object
attack              object
duration_ms        float64
flow_pkts_per_s    float64
fwd_bwd_ratio      float64
window_start        object
window_end          object
dtype: object

Colonnes numériqu

In [6]:
normal_data.to_csv("/Users/khaled/PA/Data/normal/normal_data.csv")
anomalous_data.to_csv("/Users/khaled/PA/Data/anormal/anomalous_data.csv")

✅ Étape 1 : Nettoyage + Préparation des données

In [7]:
from sklearn.preprocessing import LabelEncoder

# --- Étape 1 : Chargement des données ---
df_normal = pd.read_csv("/Users/khaled/PA/Data/normal/normal_data.csv")
df_attack = pd.read_csv("/Users/khaled/PA/Data/anormal/anomalous_data.csv")

# --- Étape 2 : Conversion des colonnes numériques ---
numeric_cols = ['total_bytes', 'pkt_count', 'psh_count', 'urg_count',
                'fwd_bytes', 'bwd_bytes', 'fwd_pkts', 'bwd_pkts',
                'dport', 'duration_ms', 'flow_pkts_per_s', 'fwd_bwd_ratio']

for col in numeric_cols:
    df_normal[col] = pd.to_numeric(df_normal[col], errors='coerce')
    df_attack[col] = pd.to_numeric(df_attack[col], errors='coerce')


In [8]:
# --- Étape 3 : Suppression des colonnes inutiles ---
cols_to_drop = ['flow_key', 'start_ts', 'end_ts', 'window_start', 'window_end']
df_normal.drop(columns=cols_to_drop, inplace=True)
df_attack.drop(columns=cols_to_drop, inplace=True)

In [9]:
# --- Étape 4 : Ajout de labels binaires (Normal vs Attack) ---
np.random.seed(42)  # reproductibilité
df_attack['attack_type'] = 'Attack'  # Tous les types d'attaque sont regroupés en une seule classe
df_normal['attack_type'] = 'Normal'

In [10]:
# Taille cible pour la classe "Attack"
taille_attack = len(df_attack)
print("Nombre d'échantillons d'attaque:", taille_attack)

# Utiliser tous les échantillons d'attaque
df_attack_equilibre = df_attack.copy()

# Vérification de l'équilibre
print(df_attack_equilibre['attack_type'].value_counts())

Nombre d'échantillons d'attaque: 69350
attack_type
Attack    69350
Name: count, dtype: int64


In [11]:
# --- Étape 5 : Équilibrage Normal vs Attack + Label Encoding ---
# Échantillonner un nombre égal d'échantillons normaux pour équilibrer avec les attaques
taille_normale = min(len(df_normal), taille_attack)
df_normal_equilibre = df_normal.sample(taille_normale, random_state=42)

print("Nombre d'échantillons normaux après équilibrage:", len(df_normal_equilibre))
print("Nombre d'échantillons d'attaque:", len(df_attack_equilibre))

# Fusion des données équilibrées
df_all = pd.concat([df_normal_equilibre, df_attack_equilibre], ignore_index=True)

# Encodage binaire (0 pour Normal, 1 pour Attack)
le = LabelEncoder()
df_all['attack_label'] = le.fit_transform(df_all['attack_type'])


Nombre d'échantillons normaux après équilibrage: 6287
Nombre d'échantillons d'attaque: 69350


In [12]:
# --- Résultat ---
print("Classes encodées (binaire) :", list(zip(le.classes_, le.transform(le.classes_))))
print("Shape finale du dataset binaire :", df_all.shape)
df_all.head()


Classes encodées (binaire) : [('Attack', np.int64(0)), ('Normal', np.int64(1))]
Shape finale du dataset binaire : (75637, 16)


,Unnamed: 0,total_bytes,pkt_count,psh_count,urg_count,fwd_bytes,bwd_bytes,fwd_pkts,bwd_pkts,dport,attack,duration_ms,flow_pkts_per_s,fwd_bwd_ratio,attack_type,attack_label
0,4441,920,12,3,0,853,67,5,7,56248,False,0.426054,11.994890,12.558824,Normal,1
1,3658,920,12,3,0,853,67,5,7,60738,False,0.620842,11.992555,12.558824,Normal,1
2,6067,920,12,3,0,853,67,5,7,42100,False,0.475883,11.994292,12.558824,Normal,1
3,5319,920,12,3,0,853,67,5,7,45556,False,0.324011,11.996113,12.558824,Normal,1
4,1539,920,12,3,0,853,67,5,7,53446,False,0.386953,11.995358,12.558824,Normal,1


In [13]:
df_all['fwd_mean_pkt_size'] = np.where(df_all['fwd_pkts']==0, 0,
                                       df_all['fwd_bytes']/df_all['fwd_pkts'])
df_all['bwd_mean_pkt_size'] = np.where(df_all['bwd_pkts']==0, 0,
                                       df_all['bwd_bytes']/df_all['bwd_pkts'])

In [14]:
df_all.to_csv("/Users/khaled/PA/Data_binaire/dataset_simule_multiclass.csv", index=False)
df_sim = pd.read_csv("/Users/khaled/PA/Data_binaire/dataset_simule_multiclass.csv")

In [15]:
UNSW_NB15_dataset = pd.read_csv("/Users/khaled/PA/Data/UNSW_NB15_training-set.csv")
len(UNSW_NB15_dataset)

175341

In [ ]:
verifier(UNSW_NB15_dataset)

In [16]:
# Afficher les catégories d'attaque originales
print("Catégories d'attaque originales dans UNSW-NB15:")
print(UNSW_NB15_dataset['attack_cat'].value_counts())

# Convertir en classification binaire (Normal vs Attack)
UNSW_NB15_dataset['attack_cat_original'] = UNSW_NB15_dataset['attack_cat'].copy()
UNSW_NB15_dataset['attack_cat'] = np.where(UNSW_NB15_dataset['attack_cat'] == 'Normal', 'Normal', 'Attack')

# Afficher la distribution binaire
print("\nDistribution binaire (Normal vs Attack):")
print(UNSW_NB15_dataset['attack_cat'].value_counts())

Catégories d'attaque originales dans UNSW-NB15:
attack_cat
Normal            56000
Generic           40000
Exploits          33393
Fuzzers           18184
DoS               12264
Reconnaissance    10491
Analysis           2000
Backdoor           1746
Shellcode          1133
Worms               130
Name: count, dtype: int64

Distribution binaire (Normal vs Attack):
attack_cat
Attack    119341
Normal     56000
Name: count, dtype: int64


In [17]:
rename_map = {
    'duration_ms'      : 'dur',
    'fwd_pkts'         : 'spkts',
    'bwd_pkts'         : 'dpkts',
    'fwd_bytes'        : 'sbytes',
    'bwd_bytes'        : 'dbytes',
    'flow_pkts_per_s'  : 'rate'
}
df_sim.rename(columns=rename_map, inplace=True)


In [18]:
# Ajout du label multiclasse (déjà présent dans df_sim["attack_type"])
df_sim['attack_cat'] = df_sim['attack_type']    # harmonisation de nom


In [19]:
# Sélection finale des 8 features + label
keep_cols = ['dur','spkts','dpkts','sbytes','dbytes','rate'
            ,'fwd_mean_pkt_size','bwd_mean_pkt_size','attack_cat']
df_sim = df_sim[keep_cols]

In [20]:
# ------------------------------------------------------------------
# Chargement des données UNSW-NB15
# ------------------------------------------------------------------
usecols_unsw = ['dur','spkts','dpkts','sbytes','dbytes','rate'
                ,'attack_cat']                # dport existe via ct_dst_sport_ltm mais -> uint16
df_unsw_tr = UNSW_NB15_dataset
# df_unsw_ts = pd.read_csv("UNSW_NB15_testing-set.csv" , usecols=usecols_unsw)
# df_unsw    = pd.concat([df_unsw_tr, df_unsw_ts], ignore_index=True)

# Assurer les mêmes types
for col in ['dur','spkts','dpkts','sbytes','dbytes','rate']:
    df_unsw_tr[col] = pd.to_numeric(df_unsw_tr[col], errors='coerce')

# Créer les features moyennes paquets pour UNSW
df_unsw_tr['fwd_mean_pkt_size'] = np.where(df_unsw_tr['spkts']==0, 0,
                                        df_unsw_tr['sbytes']/df_unsw_tr['spkts'])
df_unsw_tr['bwd_mean_pkt_size'] = np.where(df_unsw_tr['dpkts']==0, 0,
                                        df_unsw_tr['dbytes']/df_unsw_tr['dpkts'])
df_unsw_tr = df_unsw_tr[keep_cols]


In [21]:
print(len(df_unsw_tr))
print(len(df_sim))

175341
75637


In [22]:
final_data = pd.concat([df_sim, df_unsw_tr], ignore_index=True)

In [23]:
final_data

,dur,spkts,dpkts,sbytes,dbytes,rate,fwd_mean_pkt_size,bwd_mean_pkt_size,attack_cat
0,0.426054,5,7,853,67,11.994890,170.6,9.571429,Normal
1,0.620842,5,7,853,67,11.992555,170.6,9.571429,Normal
2,0.475883,5,7,853,67,11.994292,170.6,9.571429,Normal
3,0.324011,5,7,853,67,11.996113,170.6,9.571429,Normal
4,0.386953,5,7,853,67,11.995358,170.6,9.571429,Normal
...,...,...,...,...,...,...,...,...,...
250973,0.000009,2,0,114,0,111111.107200,57.0,0.000000,Attack
250974,0.505762,10,8,620,354,33.612649,62.0,44.250000,Attack
250975,0.000009,2,0,114,0,111111.107200,57.0,0.000000,Attack
250976,0.000009,2,0,114,0,111111.107200,57.0,0.000000,Attack


In [24]:
import joblib

# Vérification de la distribution des classes avant équilibrage final
print("Distribution des classes avant équilibrage final :")
print(final_data['attack_cat'].value_counts())

# Équilibrage final des classes (Normal vs Attack)
attack_samples = final_data[final_data['attack_cat'] == 'Attack']
normal_samples = final_data[final_data['attack_cat'] == 'Normal']

# Déterminer la taille cible pour l'équilibrage
target_size = min(len(attack_samples), len(normal_samples))
print(f"Taille cible pour chaque classe après équilibrage: {target_size}")


Distribution des classes avant équilibrage final :
attack_cat
Attack    188691
Normal     62287
Name: count, dtype: int64
Taille cible pour chaque classe après équilibrage: 62287


In [36]:
from imblearn.over_sampling import SMOTE

# Utiliser SMOTE pour équilibrer les classes au lieu du sous-échantillonnage
# SMOTE (Synthetic Minority Over-sampling Technique) génère des échantillons synthétiques
# pour la classe minoritaire au lieu de sous-échantillonner la classe majoritaire,
# ce qui permet de conserver toutes les informations disponibles.
# Séparer les features et la cible
X = final_data[keep_cols[:-1]]  # Toutes les colonnes sauf 'attack_cat'
y = final_data['attack_cat']

# Appliquer SMOTE pour générer des échantillons synthétiques pour la classe minoritaire
# Utiliser sampling_strategy pour contrôler le ratio des classes (pas 50/50)
# Un ratio de 0.7 signifie que la classe minoritaire sera 70% de la taille de la classe majoritaire
# Cela permet d'éviter un équilibre parfait qui pourrait être "dangereux" (surapprentissage)
sampling_ratio = 0.7  # 70% - ajuster selon les besoins
smote = SMOTE(sampling_strategy=sampling_ratio, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)


In [37]:
# Créer le dataset final avec SMOTE (intentionnellement déséquilibré)
final_balanced_data = pd.DataFrame(X_resampled, columns=keep_cols[:-1])
final_balanced_data['attack_cat'] = y_resampled

In [38]:
print(f"Taille du dataset après SMOTE: {len(final_balanced_data)} échantillons")
print(f"Distribution des classes après SMOTE:")
print(final_balanced_data['attack_cat'].value_counts())
print("Pourcentage de chaque classe après SMOTE:")
print(final_balanced_data['attack_cat'].value_counts(normalize=True) * 100)


Taille du dataset après SMOTE: 320774 échantillons
Distribution des classes après SMOTE:
attack_cat
Attack    188691
Normal    132083
Name: count, dtype: int64
Pourcentage de chaque classe après SMOTE:
attack_cat
Attack    58.823658
Normal    41.176342
Name: proportion, dtype: float64


In [39]:
# Encodage binaire
le = LabelEncoder()
final_balanced_data['attack_label'] = le.fit_transform(final_balanced_data['attack_cat'])
print("Mapping label binaire → code :", dict(zip(le.classes_, le.transform(le.classes_))))

Mapping label binaire → code : {'Attack': np.int64(0), 'Normal': np.int64(1)}


In [41]:
# Sauvegarder l'encodeur
joblib.dump(le, "models_binaire/label_encoder.joblib")

# Sauvegarder le dataset
final_balanced_data.to_csv("/Users/khaled/PA/Data_binaire/imbalanced_dataset_smote.csv", index=False)
print("Dataset préparé pour classification binaire : ", final_balanced_data.shape, "lignes, prêt pour le DL.")

# Vérification finale du dataset avec SMOTE
print("Dataset avec SMOTE sauvegardé avec succès.")
print("Note: Ce dataset est intentionnellement déséquilibré (ratio de classes ~70%) pour éviter le surapprentissage.")
print("      Il sera utilisé avec class_weight dans le modèle pour une meilleure généralisation.")

Dataset préparé pour classification binaire :  (320774, 10) lignes, prêt pour le DL.
Dataset avec SMOTE sauvegardé avec succès.
Note: Ce dataset est intentionnellement déséquilibré (ratio de classes ~70%) pour éviter le surapprentissage.
      Il sera utilisé avec class_weight dans le modèle pour une meilleure généralisation.


In [ ]:
final_data